Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
`https://github.com/squillero/computational-intelligence`  
Free for personal or classroom use; see 'LICENCE.md' for details.

In [1]:
import logging
from pprint import pprint
import numpy as np
from random import choice

logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s', datefmt='%H:%M:%S', level=logging.INFO)

# Solving sudoku with the A* algorithm


## Search algorithms

The A* algorithm is an informed search algorithm. Search algorithms take a problem as input, consisting of a goal to be achieved and a set of possible actions to be applied. They return a solution in the form of an action sequence, also known as a plan. More concretely, such algorithms solve problems starting in the initial state and for each state  explored, they check whether this is the goal state. The exploration of the nodes is done following the order given by the agenda, which is organised differently in every search algorithm. If the goal state is reached, the algorithm has finished its execution. If not, other states are considered by applying the operators to the present state. This phase is called the expansion of the state wich fills the agenda with reachable states to be explored. If at some point the agenda is empty but the goal state has not been reached yet, no solution exists for the problem. 

Below are functions (provided by professor Squillero except the "valid_sudoku" one) that will allow the algorithm, on one hand, to verify if a node is valid before trying to expand it, and, on the other hand, to check if the goal state has been reached.

In [2]:
def _contains_duplicates(X):
    return np.sum(np.unique(X)) != np.sum(X)

def contains_duplicates(sol):
    return any(_contains_duplicates(sol[r,:]) for r in range(9)) or \
           any(_contains_duplicates(sol[:,r]) for r in range(9)) or \
           any(_contains_duplicates(sol[r:r+3:,c:c+3]) for r in range(0,9,3) for c in range(0,9,3))

def valid_solution(sol):
    return not contains_duplicates(sol) and np.sum(sol) == (1+2+3+4+5+6+7+8+9) * 9

def valid_sudoku(sudoku, pos, num):
    """
    Determines if add the number num at position pos will yield a valid solution
    """
    sudoku[pos[0]][pos[1]] = num
    valid = not contains_duplicates(sudoku)
    sudoku[pos[0]][pos[1]] = 0
    return valid

def print_sudoku(sudoku):
    print("+-------+-------+-------+")
    for b in range(0, 9, 3):
        for r in range(3):
            print("|", " | ".join(" ".join(str(_) for _ in sudoku[b+r, c:c+3]) for c in range(0, 9, 3)), "|")
        print("+-------+-------+-------+")

## Informed search algorithms

For informed search methods, specific information about the state is used to generate more efficiently a solution. In the A* algoritm, the nodes to explore in its agenda will be sorted using a cost function (representing the "cost so far") and a admissible heuristic (representing the "estimated cost remaining"). 

In this implementation, a small crossed confiugration has been chosen for the heuristic meaning that the heuristic value of a cell is equal to the sum of the remaining possibilities for the row, the column and the square the cell belongs to. This is implemented mainly in the "small_cross_heuristics" function below with the support of the "reduce_possibilities" function representing the number of possibilities each cell has left regarding the numbers already present in the row, column and square of the cell.

In [3]:
def reduce_possibilities(sudoku, poss):
    """
    Reduces the number of possibilities for each cell. If a number is not possible, puts a "-1" instead.
    """
    for i in range(9):
        for j in range(9):
            if sudoku[i][j] == 0:
                poss[i][j] = np.array([p if valid_sudoku(sudoku, (i, j), p) else -1 for p in range(1, 10)])
    return poss


def small_crossed_heuristics(i, j, poss, minimal):
    """
    Returns the total number of possibilities that cell (i,j) is linked to (row, column, square)
    """
    # Possibilities of the cell
    total = len(poss[i][j][poss[i][j] >= 1]) + 1
    
    # If total < 3, interesting possibilities that we do not want to lose
    if 3 < total <= minimal: 
        total -= 1
        for k in range(9):
            if total <= minimal and k != j:
                # Row
                total += len(poss[i][k][poss[i][k] >= 1])
        for k in range(9):
            if total <= minimal and k != i:
                # Columns
                total += len(poss[k][j][poss[k][j] >= 1])
        # Get the first row of the square linked to (i,j)
        start_row = i - i % 3
        # Get the first column of the square linked to (i,j)
        start_column = j - j % 3
        # Square
        for k in range(3):
            for l in range(3):
                if total <= minimal and (k != i or l != j):
                    total += len(poss[start_row + k][start_column + l][
                                     poss[start_row + k][start_column + l] >= 1])
    return total


def update_heuristics(sudoku, h, poss):
    """
    Updates the heuristics matrix
    """
    minimal = 10000000000  # An enough high value
    for i in range(9):
        for j in range(9):
            if sudoku[i][j] == 0:
                h[i][j] = small_crossed_heuristics(i, j, poss, minimal)
                if h[i][j] < minimal:
                    minimal = h[i][j]
            else:
                h[i][j] = 0
    return h


def get_cell_to_explore(h):
    """
    Returns the best cell to explore according to the heuristics matrix
    np.amin : Takes all the minimum elements in the heuristic array, where the values are > 0
    np.where : Takes the coordinates (i, j) of all the minimum elements
    np.dstack : Unzip the result of np.where to have the coordinates in the good format
    Finally get a random cell from all the best cells
    """
    cell = choice(np.dstack(np.where(h == np.amin(h[h > 0])))[0])
    return cell

## Resolution

Below is the recursive function that will actually solve the sudoku problem using the A* algorithm.

In [4]:
def dfsolveAStar(sudoku, poss, h, a=-1, b=-1):
    """
    Implements the A* algorithm on the base of the heuristics matrix
    Fills the solution matrix by backtracking in the order given by the heuristics matrix
    """
    global num_nodes
    
    # Iterate until a valid solution is found
    if not valid_solution(sudoku):
        
        # Compute the heursitic function and chose the next node to expand
        poss = reduce_possibilities(sudoku, poss)
        h = update_heuristics(sudoku, h, poss)
        cell = get_cell_to_explore(h)

        # Try the possible numbers for the cell
        for number in poss[cell[0]][cell[1]][poss[cell[0]][cell[1]] > 0]:
            sudoku[cell[0]][cell[1]] = number
            num_nodes += 1
            # And from there try to solve the sudoku (recur)
            if dfsolveAStar(sudoku, poss, h, cell[0], cell[1]):
                return True
            
        # If expanding this node led to a wrong state, reset the cell to zero
        sudoku[cell[0]][cell[1]] = 0
        return False
    
    logging.info(f"Solved after expanding {num_nodes:,} nodes")
    return True

In [5]:
def sudoku_generator(sudokus=1, *, kappa=5, random_seed=None):
    if random_seed:
        np.random.seed(random_seed)
    for puzzle in range(sudokus):
        sudoku = np.zeros((9, 9), dtype=np.int8)
        for cell in range(np.random.randint(kappa)):
            for p, val in zip(np.random.randint(0, 8, size=(9, 2)), range(1, 10)):
                tmp = sudoku.copy()
                sudoku[tuple(p)] = val
                if contains_duplicates(sudoku):
                    sudoku = tmp
        yield sudoku.copy()

In [7]:
num_nodes = 0

for sudoku in sudoku_generator(random_seed=42):
    print_sudoku(sudoku)
    
    # Number of possibilities each cell could contain
    possibilities = np.array([[list(range(1, 10)) for _ in range(9)] for _ in range(9)])
    # Heuristic associated to each state
    heuristics = np.array([[0 for _ in range(9)] for _ in range(9)])
    
    if dfsolveAStar(sudoku, possibilities, heuristics):
        print_sudoku(sudoku)

+-------+-------+-------+
| 0 0 1 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 3 0 |
| 0 0 6 | 8 0 0 | 5 2 0 |
+-------+-------+-------+
| 9 7 0 | 4 0 0 | 0 8 0 |
| 6 0 0 | 0 3 0 | 1 0 0 |
| 0 0 0 | 0 8 6 | 0 0 0 |
+-------+-------+-------+
| 0 4 0 | 9 0 0 | 0 0 0 |
| 0 0 9 | 5 7 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+


[14:30:35] INFO: Solved after expanding 63 nodes


+-------+-------+-------+
| 2 9 1 | 3 5 7 | 8 4 6 |
| 4 8 5 | 6 9 2 | 7 3 1 |
| 7 3 6 | 8 4 1 | 5 2 9 |
+-------+-------+-------+
| 9 7 2 | 4 1 5 | 6 8 3 |
| 6 5 8 | 2 3 9 | 1 7 4 |
| 3 1 4 | 7 8 6 | 9 5 2 |
+-------+-------+-------+
| 5 4 3 | 9 6 8 | 2 1 7 |
| 1 2 9 | 5 7 3 | 4 6 8 |
| 8 6 7 | 1 2 4 | 3 9 5 |
+-------+-------+-------+
